In [1]:
import pandas as pd
import numpy as np

ANALYSIS ON IDS

In [2]:
#import data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\Season client\Season Clients Detailed_20200926-021956.csv')
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPA data\Gashora.xlsx')

In [3]:
 #Insert parameters
 Listofsectors = ['Gashora']

In [4]:
#Preliminary info on IDS data on subsidized inputs
SectorDf = IDS_data [IDS_data ['GovLocationParent'].isin(Listofsectors) ]
SubsInputsdf = SectorDf[['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_UREA kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]

#Cleaning some IDS columns
SubsInputsdf['NationalID'] =SubsInputsdf['NationalID'].astype(str).replace('\.0', '', regex=True)
SubsInputsdf['OAFID'] =SubsInputsdf['OAFID'].astype(str).replace('\.0', '', regex=True)
SubsInputsdf = SubsInputsdf.rename(columns={'GovLocationGrandParent':'GovDistrict','GovLocationParent':'GovSector','2021A_DAP kg':'DAP','2021A_NPK 17 kg':'NPK','2021A_Pan 53 kg':'PAN53', '2021A_Pan 691 kg':'PAN691', '2021A_SC 403 kg':'SC403','2021A_SC 637 kg':'SC637','2021A_UREA kg':'UREA', '2021A_WH 403 kg':'WH403','2021A_WH 505 kg':'WH505', '2021A_WH 605 kg':'WH605'})

SubsInputsdf.head()

,GovDistrict,GovSector,SiteName,LastName,FirstName,OAFID,NationalID,DAP,UREA,NPK,PAN53,PAN691,SC403,SC637,WH403,WH505,WH605
0,Bugesera,Gashora,Biryogo,BIHOYIKI,Daniel,2206,1198480141881019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bugesera,Gashora,Biryogo,BIZIMANA,Joseph,2188,nan,5.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,Bugesera,Gashora,Biryogo,DUSABIMANA,Marie jeanne,2213,1197870018581193,10.0,10.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,Bugesera,Gashora,Biryogo,HABIMANA,Damascene,2537,1198680055163043,5.0,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,Bugesera,Gashora,Biryogo,HATEGEKIMANA,Eduard,2201,1198980050525056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ANALYSIS ON MOPA

In [6]:
#Cleaning MOPA data
ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['Product'] = MOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
MOPA_dataCols = MOPA[['Name','Surname','NationalID','Product','Paid_qty']]

MOPA_table = pd.pivot_table(MOPA_dataCols, values='Paid_qty', index=['Name','Surname','NationalID'],columns=['Product'], aggfunc=np.sum).reset_index().fillna(0)

MOPA_table.head()

Product,Name,Surname,NationalID,DAP,NPK,UREA,WH403
0,ABAGIRINKA,ANNONCIATHA,1196970073902041,5.0,0.0,5.0,0.0
1,ABIMANA,THERESE,1196870059445072,5.0,0.0,5.0,0.0
2,AHOBANGEZE,CHRISTINE,1198470141927008,5.0,0.0,5.0,0.0
3,AKARUHIJE,JEAN CHRISOSTOME,1198180114522023,5.0,5.0,5.0,0.0
4,AKIMANA,BERANCILLA,1198270145874068,5.0,0.0,5.0,0.0


In [7]:
#Mactching subsidy input dataframe from season client to mopa table from SNS
IDSvsMOPA = pd.merge(SubsInputsdf,MOPA_table,on='NationalID', how='outer')

#cleaning export
IDSvsMOPA['NationalID'] = "'"+ IDSvsMOPA['NationalID']
path = r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPAvsIDS data'
os.chdir(path)
IDSvsMOPA.to_csv('Gashoratest.csv', index=False)